In [ ]:
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras
import json
import os
import fiona
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        try:
            db = create_engine('postgresql+psycopg2://'+db_user+':'+db_pw+'@'+host+'/'+default_db+'?sslmode=disable', echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

In [ ]:
db, conn = pgconnect(credentials)

In [5]:
def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(sqlcmd, args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

In [6]:
def split(word):
    if type(word)==int:
        word=str(word)
    return [char for char in word]
def toInt(st):
    parts=split(st)
    if parts.count(",")>0:
        parts.remove(",")
    i=0
    num=0
    while i<len(parts):
        num+=10**(len(parts)-i-1)* int(parts[i])
        i+=1
    return num

In [7]:
def create_wkt_element(geom, srid):
    if geom.geom_type == 'Polygon':
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid)

In [8]:
conn.execute("""
DROP TABLE  BusinessStats cascade;
CREATE TABLE BusinessStats(
   area_id INTEGER,
   area_name VARCHAR(50),
   number_of_businesses INTEGER,
   accommodation_and_food_services INTEGER,
   retail_trade INTEGER,
   agriculture_forestry_and_fishing INTEGER,
   health_care_and_social_assistance INTEGER,
   public_administration_and_safety INTEGER,
   transport_postal_and_warehousing INTEGER,
   primary key(area_id,area_name)
);""")


In [9]:
businessdata = pd.read_csv('BusinessStats.csv')
businessdata.to_sql("businessstats", con=conn, if_exists='append', index=False)
businessdata

,area_id,area_name,number_of_businesses,accommodation_and_food_services,retail_trade,agriculture_forestry_and_fishing,health_care_and_social_assistance,public_administration_and_safety,transport_postal_and_warehousing
0,101021007,Braidwood,629,26,27,280,11,0,35
1,101021008,Karabar,326,7,10,8,11,0,43
2,101021009,Queanbeyan,724,52,47,11,56,3,77
3,101021010,Queanbeyan - East,580,16,23,4,12,0,57
4,101021011,Queanbeyan Region,1642,39,63,292,34,7,81
...,...,...,...,...,...,...,...,...,...
2296,901011001,Christmas Island,0,0,0,0,0,0,0
2297,901021002,Cocos (Keeling) Islands,7,3,0,0,0,0,0
2298,901031003,Jervis Bay,6,0,3,0,0,0,0
2299,901041004,Norfolk Island,0,0,0,0,0,0,0


In [ ]:
conn.execute("""
DROP TABLE Neighbourhoods cascade;
CREATE TABLE Neighbourhoods(
    "Unnamed: 0" Integer,
   area_id INTEGER ,
   area_name VARCHAR(50),
   land_area FLOAT,
   number_of_businesses INTEGER,
   median_annual_household_income INTEGER,
   avg_monthly_rent INTEGER,
   "0-4" INTEGER,
   "5-9" INTEGER,
   "10-14" INTEGER,
   "15-19" INTEGER,
   primary key(area_id,area_name)
);""")
ls=["Unnamed: 0","area_id","area_name","land_area","number_of_businesses","median_annual_household_income","avg_monthly_rent","0-4","5-9","10-14","15-19"]
Neighbourhoodsdata = pd.read_csv('Neighbourhoods.csv',usecols=ls)
Neighbourhoodsdata.fillna(0, inplace = True)
Neighbourhoodsdata.to_sql("neighbourhoods", con=conn, if_exists='append', index=False)
population=pd.read_csv('Neighbourhoods.csv',usecols=["population"])
dwelling=pd.read_csv('Neighbourhoods.csv',usecols=["number_of_dwellings"])
population.fillna(0, inplace = True)
dwelling.fillna(0, inplace = True)


In [11]:
conn.execute("""
ALter table neighbourhoods
ADD population INTEGER
""")
conn.execute("""
ALter table neighbourhoods
ADD number_of_dwellings INTEGER
""")

In [12]:
i=0
while i<322:
    pop=toInt(population.loc[i]["population"])
    conn.execute("""
    Update neighbourhoods
    set population = {}
    where "Unnamed: 0" ={}""".format(pop,i))
    dwellings=toInt(dwelling.loc[i]["number_of_dwellings"])
    conn.execute("""
    Update neighbourhoods
    set number_of_dwellings = {}
    where "Unnamed: 0" ={}""".format(dwellings,i))
    i+=1
conn.execute("""
ALter table neighbourhoods
drop column "Unnamed: 0"
""")
query(conn,"""select * from neighbourhoods""")

OperationalError: (psycopg2.OperationalError) server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.

[SQL: 
    Update neighbourhoods
    set population = 202
    where "Unnamed: 0" =71]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
SA2_data = gpd.read_file("SA2_2016_AUST/SA2_2016_AUST.shp")
SA2_data = SA2_data.replace(to_replace='None', value=np.nan).dropna()
SA2_data

In [ ]:
SA2_og = SA2_data.copy()  # creating a copy of the original for later
SA2_data['geom'] = SA2_data['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=4283))  # applying the function
SA2_data = SA2_data.drop(columns="geometry")  # deleting the old copy
SA2_data

In [ ]:
conn.execute("""
DROP TABLE IF EXISTS sa2;
CREATE TABLE sa2 (
    "SA2_MAIN16" INTEGER primary key, 
    "SA2_5DIG16" INTEGER, 
    "SA2_NAME16" VARCHAR(50), 
    "SA3_CODE16" NUMERIC, 
    "SA3_NAME16" VARCHAR(50),
    "SA4_CODE16" NUMERIC,
    "SA4_NAME16" VARCHAR(50),
    "GCC_CODE16" VARCHAR(10),
    "GCC_NAME16" VARCHAR(50),
    "STE_CODE16" NUMERIC,
    "STE_NAME16" VARCHAR(50),
    "AREASQKM16" NUMERIC,
    geom GEOMETRY(MULTIPOLYGON,4283)
);""")

In [ ]:
SA2_data.to_sql("sa2", conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', 4283)})
query(conn, "select * from sa2")

In [ ]:
schoolsdata = pd.concat(
map(gpd.read_file, ['school_catchments/catchments_primary.shp', 'school_catchments/catchments_secondary.shp', 'school_catchments/catchments_future.shp']), ignore_index=True)
schoolsdata

In [ ]:
schoolsdata_og = schoolsdata.copy()  
schoolsdata['geom'] = schoolsdata['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=4283))  
schoolsdata = schoolsdata.drop(columns="geometry") 
schoolsdata

In [ ]:
conn.execute("""
DROP TABLE IF EXISTS schools;
CREATE TABLE schools (
    "USE_ID" NUMERIC, 
    "CATCH_TYPE" VARCHAR(20), 
    "USE_DESC" VARCHAR(50), 
    "ADD_DATE" NUMERIC, 
    "KINDERGART" VARCHAR(10),
    "YEAR1" VARCHAR(10),
    "YEAR2" VARCHAR(10),
    "YEAR3" VARCHAR(10),
    "YEAR4" VARCHAR(10),
    "YEAR5" VARCHAR(10),
    "YEAR6" VARCHAR(10),
    "YEAR7" VARCHAR(10),
    "YEAR8" VARCHAR(10),
    "YEAR9" VARCHAR(10),
    "YEAR10" VARCHAR(10),
    "YEAR11" VARCHAR(10),
    "YEAR12" VARCHAR(10),
    "PRIORITY" VARCHAR(10),
    geom GEOMETRY(MULTIPOLYGON,4283)
);"""
)

In [ ]:
schoolsdata.to_sql("schools", conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', 4283)})
query(conn, "select * from schools")

In [ ]:
breakdata = gpd.read_file("break_and_enter/BreakEnterDwelling_JanToDec2021.shp")
breakdata = breakdata.replace(to_replace='None', value=np.nan).dropna()
breakdata

In [ ]:
srid=4283
breakdata_og = breakdata.copy()  
breakdata['geom'] = breakdata['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  
breakdata = breakdata.drop(columns="geometry") 
breakdata

In [ ]:
conn.execute("""
DROP table break_enter cascade;
CREATE TABLE break_enter (
    "OBJECTID" NUMERIC primary key, 
    "Contour" NUMERIC, 
    "Density" VARCHAR(20), 
    "ORIG_FID" NUMERIC, 
    "Shape_Leng" NUMERIC,
    "Shape_Area" NUMERIC,
    geom GEOMETRY(MULTIPOLYGON,4283)
);"""
)

In [ ]:
breakdata.to_sql("break_enter", conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
query(conn, "select * from break_enter")

In [ ]:
bicycle_data = gpd.read_file('Bicycle_count_sites.geojson')
bicycle_data

In [ ]:
bicycle_data['geom'] = bicycle_data['geometry'].apply(lambda x: WKTElement(x.wkt, 4283))
bicycle_data = bicycle_data.drop(columns="geometry")
bicycle_data

In [ ]:
conn.execute("""
DROP TABLE IF EXISTS bicycles cascade;
CREATE TABLE bicycles (
    "OBJECTID" NUMERIC primary key, 
    "SiteID" NUMERIC, 
    "Intersection" VARCHAR(150),
    geom GEOMETRY(POINT,4283)
);"""
)
bicycle_data.to_sql('bicycles', conn, if_exists='append', index=False, dtype={'geom': Geometry('POINT', 4283)})
query(conn, "select * from bicycles")


In [ ]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
playground_data = gpd.read_file('Playgrounds.kml', driver='KML')
playground_data

In [ ]:
playground_data['geom'] = playground_data['geometry'].apply(lambda x: WKTElement(x.wkt, 4283))
playground_data = playground_data.drop(columns="geometry")
playground_data

In [ ]:
conn.execute("""
DROP TABLE IF EXISTS playgrounds;
CREATE TABLE playgrounds (
    "Name" VARCHAR(80), 
    "Description" VARCHAR(50), 
    geom GEOMETRY(POINT,4283)
);"""
)
playground_data.to_sql('playgrounds', conn, if_exists='append', index=False, dtype={'geom': Geometry('POINT', 4283)})
query(conn, "select * from playgrounds")

In [ ]:
conn.execute("""
DROP INDEX IF EXISTS sa2_geom_idx;
CREATE INDEX sa2_geom_idx ON sa2 USING GIST (geom);
""")
conn.execute("""
DROP INDEX IF EXISTS break_geom_idx;
CREATE INDEX break_geom_idx ON break_enter USING GIST (geom);
""")
conn.execute("""
DROP INDEX IF EXISTS school_geom_idx;
CREATE INDEX school_geom_idx ON schools USING GIST (geom);
""")
conn.execute("""
DROP INDEX IF EXISTS bicycle_geom_idx;
CREATE INDEX bicycle_geom_idx ON bicycles USING GIST (geom);
""")
conn.execute("""
DROP INDEX IF EXISTS playground_geom_idx;
CREATE INDEX playground_geom_idx ON playgrounds USING GIST (geom);
""")

In [ ]:
conn.execute("""
drop view if exists school_view cascade""")
conn.execute( """
CREATE VIEW school_view AS
SELECT A."SA2_MAIN16" as area_id, count(*), (C."0-4" + C."5-9" + C."10-14" + C."15-19") AS total, COALESCE(1000*count(*)/NULLIF((C."0-4" + C."5-9" + C."10-14" + C."15-19"), 0),0) AS school
FROM sa2 A JOIN schools B ON ST_Intersects(A.geom, B.geom)
JOIN neighbourhoods C ON A."SA2_MAIN16"=C.area_id
GROUP BY A."SA2_MAIN16", total
""")

In [ ]:
query(conn,"""
select * from school_view
""")

In [ ]:
conn.execute("""
drop view if exists bicycle_view cascade""")
conn.execute( """
CREATE VIEW bicycle_view AS
SELECT C.area_id, count(*) as bicycle_sites
FROM sa2 A JOIN bicycles B ON ST_Contains(A.geom, B.geom)
right JOIN neighbourhoods C ON A."SA2_MAIN16"=C.area_id
GROUP BY C.area_id
""")

In [ ]:
query(conn,"""
select * from bicycle_view
""")

In [ ]:
conn.execute("""
drop view if exists playground_view cascade""")
conn.execute( """
CREATE VIEW playground_view AS
SELECT C.area_id, count(*) as playground_sites
FROM sa2 A JOIN playgrounds B ON ST_Contains(A.geom, B.geom)
right JOIN neighbourhoods C ON A."SA2_MAIN16"=C.area_id
GROUP BY C.area_id
""")

In [ ]:
query(conn,"""
select * from playground_view
""")

In [ ]:
conn.execute("""
drop view if exists acc_view cascade""")
conn.execute("""
create view acc_view as
SELECT A.area_id, A.accommodation_and_food_services, B.population, COALESCE(1000*A.accommodation_and_food_services/NULLIF(B.population, 0),0) AS accom
FROM businessstats A JOIN neighbourhoods B using(area_id)
""")

In [ ]:
query(conn,"""
select * from acc_view""")

In [ ]:
conn.execute("""
drop view if exists retail_view cascade""")
conn.execute( """
create view retail_view as
SELECT A.area_id, A.retail_trade, B.population,COALESCE(1000*A.retail_trade/NULLIF(B.population, 0),0) AS retail
FROM businessstats A JOIN neighbourhoods B on A.area_id=B.area_id

""")

In [ ]:
query(conn, """
select * from retail_view""")

In [ ]:
conn.execute("""
drop view if exists health_view cascade""")
conn.execute( """
create view health_view as
SELECT A.area_id, A.health_care_and_social_assistance, B.population, COALESCE(1000*A.health_care_and_social_assistance/NULLIF(B.population, 0),0) AS health
FROM businessstats A JOIN neighbourhoods B ON A.area_id=B.area_id

""")

In [ ]:
query(conn,"""
select * from health_view""")

In [ ]:
conn.execute("""
drop view if exists crime_area cascade""")
conn.execute("""
create view crime_area as
select area_id, sum(area) as crime_area
from(
select A."SA2_MAIN16" as area_id,B."Shape_Area" as area
from sa2 A JOIN neighbourhoods C ON A."SA2_MAIN16"=C.area_id
JOIN break_enter B ON ST_Intersects(A.geom, B.geom)
order by A."SA2_MAIN16") as D
group by area_id
""")

In [ ]:
query(conn,"""
select * from crime_area
 """)

In [ ]:
conn.execute("""
drop view if exists total_area cascade""")
conn.execute("""
create view total_area as
select area_id,st_area(geom) as total_area
from sa2 join neighbourhoods on sa2."SA2_MAIN16"=neighbourhoods.area_id
order by area_id
""")
query(conn,"""
select * from total_area""")

In [ ]:
conn.execute("""
drop view if exists crime_view cascade""")
conn.execute("""
create view crime_view as
select area_id, A.crime_area ,B.total_area, COALESCE((crime_area/total_area),0) as crime
from crime_area A right join total_area B using(area_id)""")
query(conn,"""
select * from crime_view""")

In [ ]:
conn.execute("""
drop view if exists all_stats""")
conn.execute("""
create view all_stats as
select area_id,school,health,accom,crime,retail
from school_view join health_view using(area_id)
join acc_view using (area_id)
join crime_view using (area_id)
join retail_view using (area_id)
join playground_view using (area_id)
join bicycles_view using (area_id)

""")

In [ ]:
result=query(conn,"""
select * from all_stats""")
result3=query(conn,"""
select * from all_stats
where area_id in (
select "SA2_MAIN16"
from sa2
where "SA3_NAME16"='Sydney Inner City')""")
result['school_z']=(result['school']-result['school'].mean())/np.std(result['school'])
result['health_z']=(result['health']-result['health'].mean())/np.std(result['health'])
result['accom_z']=(result['accom']-result['accom'].mean())/np.std(result['accom'])
result['crime_z']=(result['crime']-result['crime'].mean())/np.std(result['crime'])
result['retail_z']=(result['retail']-result['retail'].mean())/np.std(result['retail'])
result['total_z']=result['school_z']+result['health_z']+result['accom_z']+result['retail_z']-result['crime_z']
result['score']=(np.exp(result['total_z']))/(np.exp(result['total_z'])+1)

In [ ]:
result

In [ ]:
result3['school_z']=(result3['school']-result3['school'].mean())/np.std(result3['school'])
result3['health_z']=(result3['health']-result3['health'].mean())/np.std(result3['health'])
result3['accom_z']=(result3['accom']-result3['accom'].mean())/np.std(result3['accom'])
result3['crime_z']=(result3['crime']-result3['crime'].mean())/np.std(result3['crime'])
result3['retail_z']=(result3['retail']-result3['retail'].mean())/np.std(result3['retail'])
result3['bicycle_z']=(result3['bicycle']-result3['bicycle'].mean())/np.std(result3['bicycle'])
result3['playground_z']=(result3['playground']-result3['playground'].mean())/np.std(result3['playground'])
result3['total_z']=result3['school_z']+result3['health_z']+result3['accom_z']+result3['retail_z']-result3['crime_z']+result3['bicycle_z']+result3['playground_z']
result3['score']=(np.exp(result3['total_z']))/(np.exp(result3['total_z'])+1)
result3

In [ ]:
r = np.corrcoef(result['score'],Neighbourhoodsdata['avg_monthly_rent'] )
r[1,0]

In [ ]:
r = np.corrcoef(result['score'],Neighbourhoodsdata['median_annual_household_income'] )
r[1,0]

In [ ]:
conn.close()
db.dispose()